In [2]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [3]:
from acoustic_knowledge_discovery import extractors

/home/s.kamboj.400/acoustic_knowledge_discovery/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
extractor = extractors.Extractor("/home/s.kamboj.400/acoustic_knowledge_discovery/sample_inputs/file.csv", 
                                 "/home/s.kamboj.400/acoustic_knowledge_discovery/sample_inputs/anno.csv",
                                 "/home/s.kamboj.400/acoustic_knowledge_discovery/sample_inputs/inputDtory")
kd = extractor.forward()


columns in file_ds: ['file_path', ' time_of_day', 'season', 'species']
columns in anno_ds: ['file_path', 'offset_time', 'end_time', 'species']
